In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sourmash_utils

In [3]:
import glob
import os

In [4]:
! ls -1 /mnt/data/maca-facs-sourmash_compute_all/ | head

A10-B000126-3_39_F-1-1.sig
A10-B000127-3_38_F-1-1.sig
A10-B000166-3_56_F-1-1.sig
A10-B000167-3_56_F-1-1.sig
A10-B000412-3_56_F-1-1.sig
A10-B000610-3_56_F-1-1.sig
A10-B000633-3_56_F-1-1.sig
A10-B000636-3_56_F-1-1.sig
A10-B000826-3_39_F-1-1.sig
A10-B000827-3_38_F-1-1.sig
ls: write error: Broken pipe


In [5]:
folder = '/mnt/data/maca-facs-sourmash_compute_all'

In [6]:
sourmash_utils.describe(f'{folder}/A10-B000126-3_39_F-1-1.sig')

,ksize,molecule,n_mins,name
0,21,protein,8471,cell_ontology_class:keratinocyte_stem_cell|tis...
1,21,DNA,4593,cell_ontology_class:keratinocyte_stem_cell|tis...
2,27,protein,8920,cell_ontology_class:keratinocyte_stem_cell|tis...
3,27,DNA,4769,cell_ontology_class:keratinocyte_stem_cell|tis...
4,33,protein,9175,cell_ontology_class:keratinocyte_stem_cell|tis...
5,33,DNA,4755,cell_ontology_class:keratinocyte_stem_cell|tis...
6,51,protein,8755,cell_ontology_class:keratinocyte_stem_cell|tis...
7,51,DNA,4444,cell_ontology_class:keratinocyte_stem_cell|tis...


In [ ]:
dfs = [sourmash_utils.describe(sig) for sig in glob.iglob(f'{folder}/*.sig')]
print(len(dfs))

In [ ]:
import pandas as pd

In [ ]:
descriptions_all = pd.concat(dfs)
print(descriptions_all.shape)
descriptions_all.head()

In [ ]:
import numpy as np

In [ ]:
descriptions_all['log10_n_mins'] = np.log10(descriptions_all.n_mins)
descriptions_all.head()

In [ ]:
descriptions_all.to_csv('/mnt/data/maca-facs-sourmash_compute_all_descriptions.csv')

## Write to S3!!!

In [ ]:
import s3_utils


fmt = 'parquet'
filename = f's3://olgabot-maca/facs/sourmash_compute_all_descriptions.{fmt}'

%time s3_utils.write_s3(descriptions_all, filename, fmt=fmt)

In [ ]:
# descriptions_all.mean()

In [ ]:
# descriptions_all.median()

In [ ]:
import seaborn as sns
%matplotlib inline

In [ ]:
g = sns.FacetGrid(descriptions_all, col='molecule', hue='ksize', palette='magma')
g.map(sns.kdeplot, 'n_mins', shade=False)
g.add_legend()

In [ ]:
g = sns.FacetGrid(descriptions_all, col='molecule', hue='ksize', palette='magma')
g.map(sns.kdeplot, 'log10_n_mins', shade=False)
g.add_legend()

In [ ]:
g = sns.FacetGrid(descriptions_all, col='ksize', hue='molecule', palette='magma')
g.map(sns.kdeplot, 'log10_n_mins', shade=True)
g.add_legend()

In [ ]:
bladder_droplet = pd.read_csv('/mnt/data/10X_P4_3/descriptions.csv')
bladder_droplet.head()

In [ ]:
bladder_droplet['method'] = '10x (droplet)'
descriptions_all['method'] = 'FACS (SmartSeq2)'

In [ ]:
descriptions_concat = pd.concat([bladder_droplet, descriptions_all], ignore_index=True)
print(descriptions_concat.shape)
descriptions_concat.head()

In [ ]:
%matplotlib inline

In [ ]:
g = sns.FacetGrid(descriptions_concat, col='ksize',
                   row='molecule',
                   hue='method', palette='magma')
g.map(sns.kdeplot, 'log10_n_mins', shade=False)
g.add_legend()